### User input

In [ ]:
decimal = '.'                                                                   # decimal of the input file
sep = ';'                                                                       # separator of the input file

from config import *                                                            # Personal settings of local user to set input and output directories
input_dir = input_directory + 'results/df_mediancor_sw_30.0.csv'                # input directory
output = output_directory + 'results/'                                          # output directory

headername = 'Timestamp'                                                        # header of the date  
header = 0                                                                      # header row number
dateparsingformat = '%Y-%m-%d %H:%M:%S'                                         # format of the date 
sample_name = 'sw'                                                              # name of the sample

S_f = 1                                                                         # sampling frequency
start_date = '2018-11-13 05:02:00'                                              # define the range of dates for which we 
end_date = '2018-12-03 20:44:00'                                                # want to generate the spectral curve
r2threshold = 0.98                                                              # correlation coefficient (R2) threshold to filter the negative spectral slope results
year = 'november_december_2018'                                                 # title when saving the csv files        

#To plot the time series of negative spectral slope, the user can modify:
col_sel_sc = '254.5'                                                            # column to plot
timestart = '2018-11-13 04:32:00'                                               # starting time and ending time
timeend =   '2018-12-03 20:44:00'                                
ylabel = 'Negative spectral slope [nm $^{-1}$]'                                 # label y-axis
title1 = 'spectral_curve_'                                                      # title of the exported figure 1
fig_format = '.tiff'                                                            # format of the exported figure
dpi = 300                                                                       # resolution of the exported figure


# To plot the time series of changes (%) ratio of negative spectral slope data, the user can modify:
date_ref_start = '2018-11-13 05:02:00'                                          # define reference period for computing the changes
date_ref_end = '2018-11-14 05:02:00'
ylabel2 = 'Negative spectral slope change [%]'                                  # label y-axis
title2 = 'spectral_curve_changes'                                               # title of the exported figure 2

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from datetime import datetime
from scipy import interpolate
from sklearn.linear_model import LinearRegression

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)      # make sure time column (here index) is using time format
df

### abs_spectral_curve

In [ ]:
def abs_spectral_curve(df_in, 
                       starting_date,
                       ending_date,
                       dateparsingformat,
                       sampling_frequency,
                       r2): 
    '''
    function to generate the spectral curve (get a dataframe with negative spectral slope values)
    :argument df_in: dataframe in input
    :argument starting_date: starting date
    :argument ending_date: ending date
    :argument dateparsingformat: format of the dates
    :argument sampling_frequency: sampling frequency
    :argument r2: R2 threshold
    :return: a spectral curve dataframe including columns containing only NaN (df_out1) and a spectral curve dataframe without columns containing only NaN (df_out2)
    '''
    timestamp_selected = pd.date_range(start = starting_date, end = ending_date, periods = 2).tolist() # get the index of the rows corresponding to the defined timestamps to slice the dataframe
    idx_vector = []
    df_sc = df_in.copy()
    df_sc = df_sc.reset_index()
    for timestamp in timestamp_selected:
        index_date_time = np.where(df_sc['Timestamp']==timestamp.strftime(dateparsingformat))[0] 
        idx = index_date_time[:][0]
        idx_vector.append(idx)
    ncol_in = idx_vector[0]                                         # specify that the first and second element of idx_vector are the columns where starting and ending slicing, respectively                                 
    ncol_end = idx_vector[1] 
    df_sc = df_sc.T                                                 # transpose the data frame  
    df_sc = df_sc.reset_index()                                     # restore the original index as row
    headers = df_sc.iloc[0]                                         # rename the dates as headers                                     
    df_sc = pd.DataFrame(df_sc.values[1:], columns=headers)         # convert the first row to header
    df_sc = df_sc.rename(columns={"Timestamp": "wl"})               # rename the first column as "wl" = wavelength vector
    df_sc['wl'] = df_sc['wl'].replace({'nm':''},regex=True)         # remove "nm" from the wavelength vector
    wl = df_sc['wl'].apply(pd.to_numeric)                           # convert the wavelength vector to numeric  
    if df_sc.columns.get_loc('wl') == ncol_in:                      # slice the dataframe, keeping the column "wl"
        df_sc = df_sc.iloc[:, ncol_in:ncol_end]
    else:
        df_sc = df_sc.iloc[:, np.r_[0, ncol_in:ncol_end]]                      
    headers = df_sc.columns        
    iteration = len(df_sc.columns)                                  # number of loop iterations
    print('number of iterations',iteration)
    empty_matrix = np.zeros((iteration, 3))                         # create an empty matrix to fill in with the parameters s275_295, s350_400, SR for each datetime

    interval = 21                                                   # the interval used to calculate each slope
    xx = np.arange(min(wl),max(wl),1).astype(int)
    wl_int=[]                                                       # compute number of average wavelength per interval
    for i in range(min(xx),max(xx)-interval): 
        index = (xx >= i) & (xx <= i + interval)
        wl_int.append(i + (interval / 2))
    nwl   = len(wl_int)
    ncol = len(df_sc.columns)
    ndates = ncol_end-ncol_in
    lin_reg = np.zeros((ndates,nwl))
    lin_reg_r2 = np.zeros((ndates,nwl))
    i=0
    for i in range(0,ndates, sampling_frequency):  
        print(i)
        absorbance = df_sc.iloc[:,i]                                # the absorbance vector
        #Resample data by 1 nm increment:
        xx = np.arange(min(wl),max(wl),1).astype(int)
        sf = interpolate.interp1d(wl, absorbance,kind='cubic')      # spline interpolation of third order
        yy = sf(xx)
        if min(yy)<0:    
            yy = yy-min(yy)
        xx_resh = xx.reshape((-1, 1))
        yy_log_resh = np.log(yy).reshape((-1, 1))
        yy_log_resh[yy_log_resh == float('-inf')]= np.NaN           # to replace -inf with NaN; in R "NA" says no result was available or the result is missing and it can be used in a matrix to fill in a value of a vector
        np.isnan(yy_log_resh)                                       # to see if there are NaN values
        yy=yy_log_resh[np.logical_not(np.isnan(yy_log_resh))].reshape((-1, 1)) # to remove (x, y) pairs where y is nan
        xx=xx_resh[np.logical_not(np.isnan(yy_log_resh))]
        for k, current_wl in enumerate(wl_int):             
                index = (xx >= current_wl - interval/2) & (xx <= current_wl + interval/2)  
                fit=LinearRegression().fit(xx[index].reshape((-1, 1)), yy[index]) 
                lin_reg[i,k]=-fit.coef_[0]
                lin_reg_r2[i,k] = fit.score(xx[index].reshape((-1, 1)),yy[index])
    colnames = [ str(col) for col in wl_int ]
    sc_data = pd.DataFrame(lin_reg, index=headers[int(ncol_in/(ncol_in-1)):int(ncol_end-1/ncol_end)+(ndates-1)], columns=colnames) 
    sc_data_no_zero = sc_data.loc[(sc_data!=0).any(axis=1)]
    sc_r2   = pd.DataFrame(lin_reg_r2, index=headers[int(ncol_in/(ncol_in-1)):int(ncol_end-1/ncol_end)+(ndates-1)], columns=colnames)  
    sc_r2_no_zero = sc_r2.loc[(sc_r2!=0).any(axis=1)] 
    sc_data_r2 = sc_data_no_zero[sc_r2_no_zero>=r2threshold]         # to filter the data and keep only regression with r2 >= r2threshold
    sc_data_r2 = sc_data_r2.dropna(axis=1, how='all')                # to drop the columns with only missing values
    sc_data_r2_no_NaN = sc_data_r2.dropna(axis=1, how='any')         # to drop the columns with only missing values
    sc_data_r2 = sc_data_r2.iloc[1:]
    sc_data_r2.index = pd.to_datetime(sc_data_r2.index)              # make sure time column (here index) is using time format    
    sc_data_r2.index = sc_data_r2.index.rename('Timestamp')          # rename the index column
    sc_data_r2_no_NaN = sc_data_r2_no_NaN.iloc[1:]
    sc_data_r2_no_NaN.index = pd.to_datetime(sc_data_r2_no_NaN.index)# make sure time column (here index) is using time format   
    sc_data_r2_no_NaN.index = sc_data_r2_no_NaN.index.rename('Timestamp') 
    
    df_out1 = sc_data_r2
    df_out2 = sc_data_r2_no_NaN 
    return(df_out1, df_out2)  

In [ ]:
sc_data_r2, sc_data_r2_no_NaN = abs_spectral_curve(df, start_date, end_date, dateparsingformat, S_f, r2threshold)
#sc_data_r2.to_csv(output + 'df_sc_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True, sep = ';')
#sc_data_r2_no_NaN.to_csv(output + 'df_sc_no_NaN_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True, sep = ';') # to drop the columns with any missing values

In [ ]:
%matplotlib inline # necessary if the notebook is not configured to use the inline backend by default
#%matplotlib notebook
plt.ion()
abspy.makeaplot(sc_data_r2_no_NaN, output, col_sel_sc, timestart, timeend, sample_name, title1, ylabel = 'Negative spectral slope [nm $^{-1}$]')

### Compute and visualise variation in negative spectral slope in terms of percentage changes

In [ ]:
def abs_spectral_curve_perchanges(df_in,
                                  date_ref_start,
                                  date_ref_end,
                                  dateparsingformat):
    '''
    function to compute the percentage changes of the negative spectral slope values in relation to a reference period
    :argument df_in: dataframe in input
    :argument date_ref_start: starting date of the reference period
    :argument date_ref_end: ending date of the reference period
    :argument dateparsingformat: format of the dates
    return: a dataframe with percentage changes of the negative spectral slope values in relation to a reference period
    '''
    # Define a reference period:
    date_ref_start1 = str(df_in.iloc[df_in.index.get_loc(date_ref_start, method='nearest')].name) # find the closest date to the date_ref_start and the second closest date to the date_ref_end available in the dataframe
    date_ref_end1 = str(df_in.iloc[df_in.index.get_loc(date_ref_end, method='nearest')+1].name)
    timestamp_selected = pd.date_range(start = date_ref_start1, end = date_ref_end1, periods = 2).tolist()
    idx_vector = []
    for timestamp in timestamp_selected:
            index_date_time=np.where(df_in.index == timestamp.strftime(dateparsingformat))[0] # to check for specific date-time
            idx=index_date_time[:][0]
            idx_vector.append(idx)
    #print('idx_vector:',idx_vector)

    df_ref = df_in.copy()
    df_ref = df_ref[idx_vector[0]:idx_vector[1]]

    # Normalize the data by the average of the reference period and multiply by 100 to get %change:
    df_ref_av = df_ref.mean()                          # compute the average of the vector
    df_sub_ref_av = df_in.copy()
    df_sub_ref_av = df_sub_ref_av - df_ref_av          # subtract this average to the vector
    df_change_per = (df_sub_ref_av/abs(df_ref_av))*100 # compute the percent change

    # Exclude from the dataset the reference period:
    df_change_per = df_change_per.drop(df_change_per.index[idx_vector[0]:idx_vector[1]])
    df_out = df_change_per
    return(df_out)

In [ ]:
df_sc_change_per = abs_spectral_curve_perchanges(sc_data_r2_no_NaN, date_ref_start, date_ref_end, dateparsingformat)
df_sc_change_per.to_csv(output + 'df_sc_change_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True) # export the dataframe

In [ ]:
#%matplotlib inline 
%matplotlib notebook
plt.ion()
abspy.makeaplot(df_sc_change_per, output, col_sel_sc, timestart, timeend, sample_name, title2, ylabel = 'Negative spectral slope change [%]')